View Original Code here: [Colab Notebook](https://colab.research.google.com/drive/1-5FMvszfBQiqlcKBYqssMLNsVIh0C4tw?usp=sharing)

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig

from trl import (
    SFTTrainer,
    SFTConfig,
    get_kbit_device_map,
    get_quantization_config,
    ModelConfig,
)

In [5]:
total_dataset = load_dataset("json", data_files="tweets_human_assistant_fixed.json")["train"]

split_dataset = total_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print("Sample from Train Dataset:")
for i in range(3):  # Adjust the range to view more samples
    print(train_dataset[i])

print("\nSample from Test Dataset:")
for i in range(3):  # Adjust the range to view more samples
    print(test_dataset[i])

Sample from Train Dataset:
{'text': '### Human: $AUVI Singlepoint is also in the air purification business and crazy cheap… just sayin lol 😆 $SING\n### Assistant: Positive'}
{'text': "### Human: Australia's largest broker CommSec reports that last week GameStop $GME shares were 3.8% of last weeks trades, making it #3 of most traded shares\n### Assistant: Positive"}
{'text': '### Human: $AAPL - bear flag remains in place. Failure to get over the 5 day moving average\r\n\r\n#Top5in5\n### Assistant: Negative'}

Sample from Test Dataset:
{'text': '### Human: $PPSI - Pioneer Power GAAP EPS of -$0.08 misses by $0.01, revenue of $6.04M misses by $1.62M\n### Assistant: Negative'}
{'text': '### Human: Social Media stocks catching bids alongside $TWTR’s +30% move.\r\n\r\n$FB +3.5%\r\n$SNAP +4.5%\r\n$SOCL +5.5%\r\n$PINS +11%\n### Assistant: Positive'}
{'text': '### Human: $DOGE.X up over 23% on the day 🤫\n### Assistant: Positive'}


In [7]:
model_name = "ybelkada/falcon-7b-sharded-bf16"

model_args = ModelConfig(
    model_name_or_path=model_name,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

quantization_config = get_quantization_config(model_args)

In [8]:
# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map=get_kbit_device_map(),
    use_cache=False,  # Required for gradient checkpointing
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/921M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [10]:
peft_config = LoraConfig(
    r=16,  # Low-rank adaptation parameter
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.05,  # Dropout rate for LoRA
    bias="none",  # No bias tuning
    task_type="CAUSAL_LM",  # Fine-tuning for causal language modeling
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # Target Falcon layers
)

In [11]:
peft_config.save_pretrained("./results")

In [12]:
training_args = SFTConfig(
    output_dir="./results",
    max_seq_length=256,  # Adjusted for shorter input tweets
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=200,  # Increase if the dataset is large
    num_train_epochs=5,
    warmup_ratio=0.03,
    save_steps=10,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
)

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=peft_config,
)

<ipython-input-17-bc4f645ecb10>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
trainer.train()

trainer.save_model("./results")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,13.283700
20,9.985600
30,9.431300
40,8.906900
50,8.773400
60,7.614400
70,6.977300
80,6.596000
90,5.383100
100,5.554400


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

**Testing Model Prediction with a real example:**

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("ibrahim7004/falcon-7b-finetuned-twitter", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibrahim7004/falcon-7b-finetuned-twitter", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
input_text = "### Human: yayyyy thats amazing broo\n### Assistant:"

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt")

# Remove 'token_type_ids' if it exists
inputs.pop("token_type_ids", None)

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=1,  # Limit length to prevent verbose responses
    num_return_sequences=1,  # Return only one response
    repetition_penalty=2.0,  # Penalize repeated tokens
    temperature=0.7,  # Control randomness
    top_k=2,  # Limit vocabulary to top 2 likely tokens
    top_p=0.95,  # Use nucleus sampling to focus on high-probability tokens
)

# Decode and extract response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("### Assistant:")[1].strip()  # Extract sentiment
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Positive


In [27]:
import os

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            filepath = os.path.join(dirpath, file)
            total_size += os.path.getsize(filepath)
    return total_size

# Path to the results directory
results_dir = "./results"

# Get the size in bytes, convert to MB
model_size = get_directory_size(results_dir) / (1024 * 1024)
print(f"Size of the model in './results': {model_size:.2f} MB")

Size of the model in './results': 7692.38 MB


**Uploading to HuggingFace:**

In [19]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="")

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = "./results"
hf_model_name = "ibrahim7004/falcon-7b-finetuned-twitter"

model = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

model.push_to_hub(hf_model_name)
tokenizer.push_to_hub(hf_model_name)

print(f"Model uploaded to https://huggingface.co/{hf_model_name}")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/131M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded to https://huggingface.co/ibrahim7004/falcon-7b-finetuned-twitter


----